# Project Desain Analisis Algoritma Sains Data Semester 3

### Kelompok 2

1. L0224012 - Trisha Garnis Wahyuningrun
2. L0224014 - Alvian Damar Budhi Hernowo
3. L0224037 - Talitha Sukma Mahardika

## Membuat Graph

In [10]:
import networkx as nx
import json
import random
import os

# Buat folder data jika belum ada
os.makedirs("data", exist_ok=True)

In [11]:
def generate_bem_graph(n_total, seed):
    random.seed(seed)
    n_a = int(n_total * 0.4)
    n_b = int(n_total * 0.35)
    n_c = n_total - n_a - n_b
    G_A = nx.barabasi_albert_graph(n=n_a, m=1, seed=seed)
    G_B = nx.barabasi_albert_graph(n=n_b, m=1, seed=seed+1)
    G_C = nx.barabasi_albert_graph(n=n_c, m=1, seed=seed+2)
    G_total = nx.disjoint_union_all([G_A, G_B, G_C])
    node_data = {}
    nama = []
    file = open("data/nama.txt","a+")
    file.seek(0)
    for i in file.readlines() :
        nama.append(i)
    file.close()
    nnama = len(nama) - 1
    for node_id in G_total.nodes():
        if node_id < n_a:
            divisi = "Divisi Acara"
            jabatan = "Ketua Divisi" if node_id == 0 else "Staff"
        elif node_id < (n_a + n_b):
            divisi = "Divisi Humas"
            jabatan = "Ketua Divisi" if node_id == n_a else "Staff"
        else:
            divisi = "Divisi Logistik"
            jabatan = "Ketua Divisi" if node_id == (n_a + n_b) else "Staff"
        nama1 = nama.pop(random.randint(0,nnama)).strip()
        nama2 = nama.pop(random.randint(0,nnama-1)).strip()
        nnama -= 2
        node_data[str(node_id)] = {
            "nama": nama1 + " " + nama2,
            "divisi": divisi,
            "jabatan": jabatan,
            "ipk": round(random.uniform(2.5, 4.0), 2)
        }
    adj_list = nx.to_dict_of_lists(G_total)
    
    return adj_list, node_data

In [12]:
n = 375 
seed = 121437 
adj_list, node_data = generate_bem_graph(n, seed)      
output_content = {
    "project": "connected_components_social_graph",
    "description": f"Graf BEM {n} nodes dengan Data Atribut",
    "n_nodes": n,
    "graph_adj": adj_list,
    "node_data": node_data
}       
filename = f"data/social_graph_N{n}.json"
with open(filename, "w") as f:
    json.dump(output_content, f, indent=2)           
print(f"[OK] Berhasil membuat: {filename} (Lengkap dengan data mahasiswa)")

[OK] Berhasil membuat: data/social_graph_N375.json (Lengkap dengan data mahasiswa)
